<a href="https://colab.research.google.com/github/fknbtrd/netology_homework/blob/main/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0%20%D0%B2%20Python%20%E2%80%94%20%D0%9F%D0%BE%D0%B8%D1%81%D0%BA%20%D0%B0%D0%BD%D0%BE%D0%BC%D0%B0%D0%BB%D0%B8%D0%B9%20%D0%B2%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B8%20%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%20%D1%81%20%D0%BF%D1%80%D0%BE%D0%BF%D1%83%D1%89%D0%B5%D0%BD%D0%BD%D1%8B%D0%BC%D0%B8%20%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np


# ЗАДАНИЕ 1: ЗАГРУЗКА ДАННЫХ И ВЫБОР СТОЛБЦОВ

# Чтение данных (предполагаем, что файл называется 'horse_data.csv')
df = pd.read_csv('horse_data.csv', header=None)

# Выбор нужных столбцов (первые 8) и присвоение имен
column_names = [
    'surgery?', 'Age', 'rectal temperature', 'pulse',
    'respiratory rate', 'temperature of extremities', 'pain', 'outcome'
]
df = df.iloc[:, :8]
df.columns = column_names

# Замена "?" на NaN и преобразование типов
df.replace('?', np.nan, inplace=True)

# Преобразование числовых столбцов
numeric_cols = ['rectal temperature', 'pulse', 'respiratory rate']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Преобразование категориальных столбцов
categorical_cols = ['surgery?', 'Age', 'temperature of extremities', 'pain', 'outcome']
df[categorical_cols] = df[categorical_cols].astype('category')

print(f"Размер датафрейма: {df.shape}")
print("Столбцы:", df.columns.tolist())



# ЗАДАНИЕ 2: ПЕРВИЧНОЕ ИЗУЧЕНИЕ ДАННЫХ

# 2.1. Базовые статистики для числовых столбцов
print("\n1. Базовые статистики для числовых столбцов:")
print(df[numeric_cols].describe().round(2))

# 2.2. Анализ категориальных столбцов
print("\n2. Распределение категориальных значений:")
for col in categorical_cols:
    print(f"\nСтолбец '{col}':")
    print(df[col].value_counts(dropna=False))

# 2.3. Поиск выбросов в числовых столбцах
print("\n3. Анализ выбросов (значения за пределами Q1-1.5*IQR и Q3+1.5*IQR):")
for col in numeric_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    print(f"\nСтолбец '{col}':")
    print(f"Границы нормальных значений: [{lower_bound:.2f}, {upper_bound:.2f}]")
    print(f"Количество выбросов: {len(outliers)}")
    if len(outliers) > 0:
        print("Примеры выбросов:", outliers.head(3).tolist())


# ЗАДАНИЕ 3: РАБОТА С ПРОПУСКАМИ

# 3.1. Анализ пропущенных значений
print("\n1. Анализ пропущенных значений:")
missing_data = pd.DataFrame({
    'Количество пропусков': df.isnull().sum(),
    'Процент пропусков': (df.isnull().mean() * 100).round(2)
})
print(missing_data)

# 3.2. Стратегии заполнения пропусков
print("\n2. Стратегии заполнения пропусков:")

# Копируем датафрейм для заполнения
df_filled = df.copy()

# Для категориальных столбцов используем моду (наиболее частое значение)
for col in categorical_cols:
    mode_val = df[col].mode()[0]
    df_filled[col] = df_filled[col].fillna(mode_val)
    print(f"Категориальный столбец '{col}': заполнено {df[col].isnull().sum()} пропусков значением '{mode_val}'")

# Для числовых столбцов используем медиану (устойчива к выбросам)
for col in numeric_cols:
    median_val = df[col].median()
    df_filled[col] = df_filled[col].fillna(median_val)
    print(f"Числовой столбец '{col}': заполнено {df[col].isnull().sum()} пропусков медианой {median_val:.2f}")

# 3.3. Проверка, что пропусков не осталось
print("\n3. Проверка пропусков в итоговом датафрейме:")
print("Осталось пропусков:", df_filled.isnull().sum().sum())

# 3.4. Сохранение результата
df_filled.to_csv('horse_data_processed.csv', index=False)
print("\nОбработанные данные сохранены в файл 'horse_data_processed.csv'")

Размер датафрейма: (300, 8)
Столбцы: ['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']

1. Базовые статистики для числовых столбцов:
       rectal temperature   pulse  respiratory rate
count              300.00  240.00            276.00
mean           1085888.83   38.17             71.91
std            1529800.91    0.73             28.63
min             518476.00   35.40             30.00
25%             528904.00   37.80             48.00
50%             530305.50   38.20             64.00
75%             534727.50   38.50             88.00
max            5305629.00   40.80            184.00

2. Распределение категориальных значений:

Столбец 'surgery?':
surgery?
1      180
2      119
NaN      1
Name: count, dtype: int64

Столбец 'Age':
Age
1    276
9     24
Name: count, dtype: int64

Столбец 'temperature of extremities':
temperature of extremities
NaN    58
20     28
24     27
16     22
12     19
30     19
40     